# 1. 강의영상 

{{<video https://youtu.be/playlist?list=PLQqh36zP38-ziuAKfEqwmaDeJdCNGuLM5&si=D_zP0P21Di0uePBD >}}

# 2. Imports 

In [1]:
import numpy as np
import pandas as pd
import sklearn.linear_model
import itertools

# 3. Data 에 대한 정리

`-` 주어진 자료가 아래와 같다고 하자. 

In [2]:
df = pd.DataFrame({'X':np.arange(20,30),'y':-np.arange(10)+1+np.random.randn(10)*0.1})
df

,X,y
0,20,1.159315
1,21,0.011628
2,22,-0.913198
3,23,-1.953295
4,24,-3.138916
5,25,-3.976391
6,26,-5.163496
7,27,-5.859210
8,28,-7.174108
9,29,-7.908619


`-` 훈련에는 자료 0~7까지 사용하고 테스트에는 자료 8,9를 사용하기로 하자. 

In [4]:
df_train = df[:8]
df_test = df[8:]

In [5]:
df_train

,X,y
0,20,1.159315
1,21,0.011628
2,22,-0.913198
3,23,-1.953295
4,24,-3.138916
5,25,-3.976391
6,26,-5.163496
7,27,-5.859210


In [6]:
df_test

,X,y
8,28,-7.174108
9,29,-7.908619


`-` 아래와 같은 자료를 정리하자.

In [7]:
df_train_X = df_train[['X']]
df_train_y = df_train[['y']]
df_test_X = df_test[['X']]
df_test_y = df_test[['y']]

# 4. Predictor 에 대하여

## A. 학습 이후에 예측/평가 가능

`-` Predictor의 list생성 

In [9]:
[sklearn.linear_model.LinearRegression(), sklearn.linear_model.LinearRegression()]

[LinearRegression(), LinearRegression()]

In [10]:
predictors = [sklearn.linear_model.LinearRegression() for i in range(2)]
predictors

[LinearRegression(), LinearRegression()]

- 두개의 predictor를 만들어서 리스트로 정리함

`-` 첫번째 predictor에 접근

In [11]:
predictors[0]

LinearRegression()

`-` 두번째 predictor에 접근

In [17]:
predictors[1]

LinearRegression()

`-` 첫번째 predictor를 학습

In [18]:
predictors[0].fit(df_train_X,df_train_y)

LinearRegression()

`-` 학습이후에는 `.coef_`, `.intercept_` 값이 생성됨 

In [19]:
predictors[0].coef_, predictors[0].intercept_

(array([[-1.01643449]]), array([21.40701511]))

In [20]:
predictors[1].coef_, predictors[1].intercept_

AttributeError: 'LinearRegression' object has no attribute 'coef_'

`-` `.coef_`와 `.intercept_`값이 생겨야 `.predict(X)`를 통하여 예측을 할 수 있음 

In [25]:
predictors[0].predict(df_train_X)

array([[ 1.07832538],
       [ 0.0618909 ],
       [-0.95454359],
       [-1.97097808],
       [-2.98741256],
       [-4.00384705],
       [-5.02028154],
       [-6.03671602]])

In [28]:
predictors[1].predict(df_train_X) # 학습이 안되어있으므로 당연히 에러!

NotFittedError: This LinearRegression instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.

`-` 예측을 해야 평가를 할 수 있음 

In [29]:
predictors[0].score(df_train_X,df_train_y)

0.9980028392074787

In [30]:
predictors[1].score(df_train_X,df_train_y)

NotFittedError: This LinearRegression instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.

## B. `.fit(X,y)`에서 `X`,`y`의 형식

In [36]:
predictr = predictors[0]
predictr

LinearRegression()

In [52]:
predictr.fit(df_train_X, np.array(df_train_y).reshape(-1)) # 이건 잘 됨.

LinearRegression()

In [60]:
df_train_X.shape, np.array(df_train_X).reshape(-1).shape

((8, 1), (8,))

In [86]:
np.array(df_train_X).tolist(), np.array(df_train_X).reshape(-1).tolist()

([[20], [21], [22], [23], [24], [25], [26], [27]],
 [20, 21, 22, 23, 24, 25, 26, 27])

In [55]:
predictr.fit(np.array(df_train_X).reshape(-1), np.array(df_train_y).reshape(-1)) # 이건 안됨!

ValueError: Expected 2D array, got 1D array instead:
array=[20 21 22 23 24 25 26 27].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.

`-` 어떤 경우에 에러가 나고 안나는지 관찰해보자.

In [59]:
Xs = {'DataFrame(2d)': df_train_X, 
      'Seires(1d)': df_train_X.X,
      'ndarray(2d)': np.array(df_train_X),
      'ndarray(1d)': np.array(df_train_X).reshape(-1),
      'list(2d)': np.array(df_train_X).tolist(),
      'list(1d)': np.array(df_train_X).reshape(-1).tolist()}

In [61]:
ys = {'DataFrame(2d)': df_train_y, 
      'Seires(1d)': df_train_y.y,
      'ndarray(2d)': np.array(df_train_y),
      'ndarray(1d)': np.array(df_train_y).reshape(-1),
      'list(2d)': np.array(df_train_y).tolist(),
      'list(1d)': np.array(df_train_y).reshape(-1).tolist()}

In [62]:
def test(X,y):
    try: 
        predictr.fit(X,y)
        return 'no error'
    except: # 예외상황 발생.
        return 'error'

In [63]:
X = np.array(df_train_X).reshape(-1)
y = np.array(df_train_y).reshape(-1)

In [64]:
test(X,y)

'error'

In [168]:
{('X='+i,'y='+j): test(Xs[i],ys[j]) for i,j in itertools.product(Xs.keys(),ys.keys())}

{('X=DataFrame(2d)', 'y=DataFrame(2d)'): 'no error',
 ('X=DataFrame(2d)', 'y=Seires(1d)'): 'no error',
 ('X=DataFrame(2d)', 'y=ndarray(2d)'): 'no error',
 ('X=DataFrame(2d)', 'y=ndarray(1d)'): 'no error',
 ('X=DataFrame(2d)', 'y=list(2d)'): 'no error',
 ('X=DataFrame(2d)', 'y=list(1d)'): 'no error',
 ('X=Seires(1d)', 'y=DataFrame(2d)'): 'error',
 ('X=Seires(1d)', 'y=Seires(1d)'): 'error',
 ('X=Seires(1d)', 'y=ndarray(2d)'): 'error',
 ('X=Seires(1d)', 'y=ndarray(1d)'): 'error',
 ('X=Seires(1d)', 'y=list(2d)'): 'error',
 ('X=Seires(1d)', 'y=list(1d)'): 'error',
 ('X=ndarray(2d)', 'y=DataFrame(2d)'): 'no error',
 ('X=ndarray(2d)', 'y=Seires(1d)'): 'no error',
 ('X=ndarray(2d)', 'y=ndarray(2d)'): 'no error',
 ('X=ndarray(2d)', 'y=ndarray(1d)'): 'no error',
 ('X=ndarray(2d)', 'y=list(2d)'): 'no error',
 ('X=ndarray(2d)', 'y=list(1d)'): 'no error',
 ('X=ndarray(1d)', 'y=DataFrame(2d)'): 'error',
 ('X=ndarray(1d)', 'y=Seires(1d)'): 'error',
 ('X=ndarray(1d)', 'y=ndarray(2d)'): 'error',
 ('X=n

`-` 결론: X는 2d만 가능, y는 2d,1d 모두 가능

`-` itertools 사용법 (참고)

(예제1)

In [67]:
for i,j in itertools.product(['A','B'], [1,2,3]): # iterable object
    print(i,j)

A 1
A 2
A 3
B 1
B 2
B 3


(예제2)

In [75]:
X = np.array(df_train_X).reshape(-1)
y = np.array(df_train_y).reshape(-1)

Xs = {'DataFrame(2D)':[1,2,3], 'Series(1d)':[2,3,4]}
ys = {'A':1, 'B':2, 'C':3}

In [76]:
for i,j in itertools.product(Xs.keys(), ys.keys()):
    print(i,j)

DataFrame(2D) A
DataFrame(2D) B
DataFrame(2D) C
Series(1d) A
Series(1d) B
Series(1d) C


In [77]:
for i,j in itertools.product(Xs.keys(), ys.keys()):
    print(Xs[i],ys[j])

[1, 2, 3] 1
[1, 2, 3] 2
[1, 2, 3] 3
[2, 3, 4] 1
[2, 3, 4] 2
[2, 3, 4] 3


(예제3)

In [78]:
Xs = {'DataFrame(2d)':df_train_X,
      'Series(1d)': df_train_X.X}

ys = {'DataFrame(2d)': df_train_y,
      'Series(1d)': df_train_y.y}

In [83]:
for i,j in itertools.product(Xs, ys):
    print(Xs[i], ys[j])

    X
0  20
1  21
2  22
3  23
4  24
5  25
6  26
7  27           y
0  1.159315
1  0.011628
2 -0.913198
3 -1.953295
4 -3.138916
5 -3.976391
6 -5.163496
7 -5.859210
    X
0  20
1  21
2  22
3  23
4  24
5  25
6  26
7  27 0    1.159315
1    0.011628
2   -0.913198
3   -1.953295
4   -3.138916
5   -3.976391
6   -5.163496
7   -5.859210
Name: y, dtype: float64
0    20
1    21
2    22
3    23
4    24
5    25
6    26
7    27
Name: X, dtype: int64           y
0  1.159315
1  0.011628
2 -0.913198
3 -1.953295
4 -3.138916
5 -3.976391
6 -5.163496
7 -5.859210
0    20
1    21
2    22
3    23
4    24
5    25
6    26
7    27
Name: X, dtype: int64 0    1.159315
1    0.011628
2   -0.913198
3   -1.953295
4   -3.138916
5   -3.976391
6   -5.163496
7   -5.859210
Name: y, dtype: float64


In [80]:
for i,j in itertools.product(Xs, ys):
    print(test(Xs[i], ys[j]))

no error
no error
error
error


# 5. Notations에 대한 우리끼리의 약속 

`-` `df_train_X`는 흔하게 쓰는 표현임. 이는 아래의 점에서 우수함

- 형식이 df임이 명시되어 있음. 
- train에 사용된다는 것이 명시되어있음. 
- 설명변수라는 것이 명시되어 있음. 

`-` 그렇지만 이러한 식으로 한다면 아래와 같은 이름들을 사용해야함

- `df_train_X`
- `nparray2d_train_X`
- `list2d_train_X`

이것은 너무 복잡하므로 앞으로 이러한 경우를 모두 합쳐서 `X` 라고 표현할 것임 

`-` 마찬가지의 논리로 아래와 같은 기호를 약속하자. 

- `X`: `df_train_X`, `ndarray2d_train_X`,`list2d_train_X`
- `y`: `df_train_y`, `series_train_y`, ... , `list_2d_train_y`
- `XX`: `df_test_X`, `ndarray2d_test_X`,`list2d_test_X`
- `yy`: `df_test_y`, `series_test_y`, ... , `list_2d_test_y`

`-` 또한 아래와 같은 기호를 약속하자. 

- `yhat`: `predictr.predict(X)`
- `yyhat`: `predictr.predict(XX)`

`-` 참고: 일반적으로 `predictor`가 기대하는 형태는 아래와 같음 

In [68]:
X = np.array(df_train_X) # 2d numpy array형태를 기대
y = np.array(df_train_y).reshape(-1) # 1d numpy array를 기대

In [69]:
X,y

(array([[20],
        [21],
        [22],
        [23],
        [24],
        [25],
        [26],
        [27]]),
 array([ 1.15931539,  0.01162796, -0.91319829, -1.95329462, -3.13891577,
        -3.97639123, -5.16349632, -5.85920968]))

In [70]:
predictr.fit(X,y)

LinearRegression()

In [71]:
predictr.predict(X) # 이게 원래 기대하는 형태임

array([ 1.07832538,  0.0618909 , -0.95454359, -1.97097808, -2.98741256,
       -4.00384705, -5.02028154, -6.03671602])